# Exploring the gemma model

Exploratory analyses by Casey Dunn, [https://dunnlab.org]. I am running these analyses locally on a Windows 11 machine with CUDA 11.8.

The goal is to take a look under the hood of the [Gemma](https://www.kaggle.com/models/google/gemma) model to better understand [transformers](https://arxiv.org/abs/1706.03762). In particular, I would like to have a practical understanding of the concepts explored in Grant Sanderson's series starting [here](https://youtu.be/wjZofJX0v4M?si=-Lsf1nH69cNyNVi3).

## Importing model weights

[Download](https://www.kaggle.com/models/google/gemma) the model weights for pytorch. This requires that you create a kaggle account, and you may need to request access to the model (approval can take a day or two). 

I used the  the 1.1-2b-it variation, and downloaded them as a as a `tar.gz` file. Decompress the file to get the `gemma-2b-it.ckpt` model weights file.

## Configuring your environment

Install the necessary packages

```bash
nvcc --version
conda create --name torch
conda activate torch
conda install pytorch torchvision torchaudio pytorch-cuda=11.8 immutabledict sentencepiece matplotlib -c pytorch -c nvidia -c conda-forge
```

## Loading the model


In [ ]:
import torch

from gemma import config
from gemma import model as gemma_model
import random
import numpy as np
import contextlib

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())

Using code from https://ai.google.dev/gemma/docs/pytorch_gemma/ to implement the Gemma model in PyTorch. Also drawing from https://github.com/google/gemma_pytorch/blob/main/scripts/run.py

In [ ]:
# Choose variant and machine type
VARIANT = '2b' # also works for '2b-it'
MACHINE_TYPE = 'cuda'
CKPT_PATH="C:\\Users\\casey\\Downloads\\gemma-2b-it.ckpt"

In [ ]:
model_config = config.get_model_config(VARIANT)
model_config.dtype = "float32" if MACHINE_TYPE == "cpu" else "float16"
model_config.quant = False

# Seed random.
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
@contextlib.contextmanager
def set_default_tensor_type(dtype: torch.dtype):
    """Sets the default torch dtype to the given dtype."""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

device = torch.device(MACHINE_TYPE)
with set_default_tensor_type(model_config.get_dtype()):
    model = gemma_model.GemmaForCausalLM(model_config)
    model.load_weights(CKPT_PATH)
    model = model.to(device).eval()
print("Model loading done")

In [ ]:
# summarize the structure of the model
print(model)

In [ ]:
## Testing the model

In [ ]:
prompt = "What are maxwell's equations?"
output_len = 200
result = model.generate(prompt, device, output_len=output_len)
print(result)

## Exploring the tokenizer

The tokenizer converts the input text into a sequence of integers, each representing a token. Sometimes a token is a whole word, sometimes it is a subword. A fun look at tokenizers can be found at [https://youtu.be/eLDYPvc7DNA?si=WPcVyjdCQlMe5cU0].

In [ ]:
# Access the tokenizer from the model
tokenizer = model.tokenizer

# Input string
input_string = "This is an example sentence."

# Tokenize the input string
tokens = tokenizer.encode(input_string)

# Print the resulting tokens
print(tokens)

In [ ]:
for token in tokens:
  print(f"\t{token}\t{tokenizer.decode(token)}")

In [ ]:
decoded = tokenizer.decode(tokens)
print(decoded)

In [ ]:
print(tokenizer.encode("is"))
print(tokenizer.encode(" is"))
print(tokenizer.encode("Is"))

In [ ]:
def show_token_breaks(tokenizer, text):
	tokens = tokenizer.encode(text)
	tokens_text = list()
	for token in tokens:
		token_text = tokenizer.decode(token)
		tokens_text.append(token_text)
	return "|".join(tokens_text)

Rather than look at lists of token integers and text, we can just mark the breaks between the tokens in the text with `|`.

In [ ]:
print(show_token_breaks(tokenizer, "This is an example sentence."))

In [ ]:
print(show_token_breaks(tokenizer, "The gastrozooids of siphonophores are a type of polyp."))

In [ ]:
print(show_token_breaks(tokenizer, "let x_squared = x * x;"))

In [ ]:
print(show_token_breaks(tokenizer, "y=mx+b"))

## Exploring the embedder

The embedder provides a vector embedding of each token. These vectors are the input to the transformer of the llm. 

First, let's copy the embeddings out of the model. 

In [ ]:
embeddings = model.embedder.weight.data

# Move embeddings to the desired device (optional)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embeddings = embeddings.to(device)

# Print the shape of the embeddings tensor
print(embeddings.shape)

We see that there are are 256000 unique tokens, each represented as a vector in $\mathbb{R}^2048$ (2048 dimensional space).

In [ ]:
# Calculate the euclidean lengths of the embeddings
lengths = torch.linalg.norm(embeddings, ord=2, dim=1)
lengths.shape
print(lengths[0:20])

# Plot a histogram of the lengths
import matplotlib.pyplot as plt
plt.hist(lengths.cpu().numpy(), bins=50)
plt.show()

This is a histogram of the lengths of all the embeddings. We can see that the embeddings are not all the same lengths, ie they do not fall on a sphere in $\mathbb{R}^{2048}$. This means that embeddings differ by both the angle and the magnitude of the vector.

This means that the cosine distance and the euclidean distance between embeddings carry different (though of course related) information. So we will examine both.

In [ ]:
def find_closest_tokens_by_embedding(embeddings, embedding, n=5, cosine=True):
	if cosine:
		# Calculate the cosine similarities to all other embeddings
		similarities = torch.nn.functional.cosine_similarity(embeddings, embedding.unsqueeze(0), dim=1)
		# Get the indices of the n largest similarities
		closest_tokens = torch.argsort(similarities, descending=True)[:n]
	else:
		# Calculate the euclidean distances to all other embeddings
		distances = torch.linalg.norm(embeddings - embedding, ord=2, dim=1)
		# Get the indices of the n smallest distances
		closest_tokens = torch.argsort(distances)[:n]
	return closest_tokens


# Function to find the n closest tokens to a given token based on euclidean distance in the embedding space
def find_closest_tokens(embeddings, token, n=5, cosine=True):
	# Get the embedding for the token
	embedding = embeddings[token]
	
	# Calculate the euclidean distances to all other embeddings
	closest_tokens = find_closest_tokens_by_embedding(embeddings, embedding, n, cosine)
	return closest_tokens

Let's look at the embeddings in the neighborhood of the token for `chicken`.

In [ ]:
chicken_token = tokenizer.encode("chicken")[1]
print(chicken_token)

First, here are the closest tokens according to cosine similarity.

In [ ]:
# Find the 5 closest tokens to the chicken token
closest_tokens = find_closest_tokens(embeddings, chicken_token, n=20, cosine=True)
print(closest_tokens)

# Convert the tokens to text
closest_tokens_text = [tokenizer.decode(int(token)) for token in closest_tokens]
print(closest_tokens_text)

Now we look at the closest tokens according to euclidean distance.

In [ ]:
# Find the 5 closest tokens to the chicken token
closest_tokens = find_closest_tokens(embeddings, chicken_token, n=20, cosine=False)
print(closest_tokens)

# Convert the tokens to text
closest_tokens_text = [tokenizer.decode(int(token)) for token in closest_tokens]
print(closest_tokens_text)

In [ ]:
token2 = tokenizer.encode("house")[1]
print(token2)

In [ ]:
# Find the 5 closest tokens to the chicken token
closest_tokens = find_closest_tokens(embeddings, token2, n=20, cosine=True)
print(closest_tokens)

# Convert the tokens to text
closest_tokens_text = [tokenizer.decode(int(token)) for token in closest_tokens]
print(closest_tokens_text)

In [ ]:
# Find the 5 closest tokens to the chicken token
closest_tokens = find_closest_tokens(embeddings, token2, n=20, cosine=False)
print(closest_tokens)

# Convert the tokens to text
closest_tokens_text = [tokenizer.decode(int(token)) for token in closest_tokens]
print(closest_tokens_text)

The cosine similarity might be a bit more interpretable than euclidean distance, and seems to be the norm in the field, so we will use that for now.

## Analogy engine

Each dimension in the embedding space can correspond to a different feature of the tokens. We can examine these relationships by subtracting the embeddings of two tokens, and applying that difference to another word. This is basically an analogy engine. For example, if we find the embedding of `king` - `man` + `woman`, we might get an embedding that is close to `queen`.

In [ ]:
def analogy_by_tokens(embeddings, token_a, token_b, token_c, n=5):
	# Get the embeddings for the tokens
	embedding_a = embeddings[token_a]
	embedding_b = embeddings[token_b]
	embedding_c = embeddings[token_c]
	
	# Calculate the analogy vector
	analogy_vector = embedding_b - embedding_a + embedding_c
	
	# Calculate the euclidean distances to all other embeddings
	closest_tokens = find_closest_tokens_by_embedding(embeddings, analogy_vector, n)
	return closest_tokens

def analogy(embeddings, tokenizer, text_a, text_b, text_c, n=5):
	# Encode the text to tokens
	token_a = tokenizer.encode(text_a)[1]
	token_b = tokenizer.encode(text_b)[1]
	token_c = tokenizer.encode(text_c)[1]
	
	# Calculate the analogy
	closest_tokens = analogy_by_tokens(embeddings, token_a, token_b, token_c, n)
	
	# Convert the tokens to text
	closest_tokens_text = [tokenizer.decode(int(token)) for token in closest_tokens]

	return closest_tokens_text

def token_distance(embeddings, token_a, token_b):
	embedding_a = embeddings[token_a]
	embedding_b = embeddings[token_b]
	distance = torch.linalg.norm(embedding_a - embedding_b, ord=2)
	return distance

def text_distance(embeddings, tokenizer, text_a, text_b):
	token_a = tokenizer.encode(text_a)[1]
	token_b = tokenizer.encode(text_b)[1]
	distance = token_distance(embeddings, token_a, token_b)
	return distance

In [ ]:
print(analogy(embeddings, tokenizer, "man", "king", "woman", 20))

In [ ]:
print(analogy(embeddings, tokenizer, "man", "woman", "uncle", 20))

In [ ]:
print(analogy(embeddings, tokenizer, "finger", "hand", "petal", 20))

In [ ]:
print(analogy(embeddings, tokenizer, "fire", "burn", "flood", 20))

In [ ]:
print(analogy(embeddings, tokenizer, "apple", "apples", "banana", 20))

In [ ]:
print(text_distance(embeddings, tokenizer, "man", "woman"))

On the whole, the analogy machine isn't great. It may be that without any sort of refinement from an attention mechanism, the raw embeddings are not so informative. King could mean a card, for example.

# Local neighborhoods

Above we just look at what tokens are nearby. Now let's look at the local neighborhood of a token in the embedding space. We will use PCA to reduce the dimensionality of the embeddings to 2D, and then plot the neighborhood of a token in this 2D space.

In [ ]:
# Function that given a token and embeddings, finds the n closest tokens based on cosine similarity.
# It then returns a tuple with:
# - A vector where the first element is the original token and the rest are the closest tokens.
# - The subset of the embeddings corresponding to the tokens.
def find_closest_tokens_by_cosine(embeddings, token, n=5):
	# Get the embedding for the token
	embedding = embeddings[token]
	
	# Calculate the cosine similarities to all other embeddings
	similarities = torch.nn.functional.cosine_similarity(embeddings, embedding.unsqueeze(0), dim=1)
	
	# Get the indices of the n largest similarities
	closest_tokens = torch.argsort(similarities, descending=True)[:n]
	
	# Create a vector where the first element is the original token and the rest are the closest tokens
	closest_tokens_vector = torch.cat([torch.tensor([token]), closest_tokens])
	
	# Get the subset of the embeddings corresponding to the tokens
	closest_embeddings = embeddings[closest_tokens_vector]
	return closest_tokens_vector, closest_embeddings

# Function that given embeddings, runs a PCA on the embeddings and returns the first two principal components.
def run_pca(embeddings):
	# Calculate the mean of the embeddings
	mean = torch.mean(embeddings, dim=0)
	
	# Subtract the mean from the embeddings
	centered = embeddings - mean
	
	# Calculate the covariance matrix
	cov = torch.matmul(centered.T, centered)
	
	# Calculate the eigenvalues and eigenvectors of the covariance matrix
	eigenvalues, eigenvectors = torch.linalg.eig(cov)
	
	# Get the indices of the eigenvalues sorted in descending order
	indices = torch.argsort(eigenvalues[:, 0], descending=True)
	
	# Get the first two principal components
	pc1 = eigenvectors[:, indices[0]]
	pc2 = eigenvectors[:, indices[1]]
	
	return pc1, pc2

# Function that given a token and embeddings, runs find_closest_tokens_by_cosine() and run_pca() and plots the results.
# tokens are converted to text using the tokenizer and added to the plot.
def plot_closest_tokens_by_cosine(embeddings, tokenizer, token, n=5):
	# Find the closest tokens by cosine similarity
	closest_tokens_vector, closest_embeddings = find_closest_tokens_by_cosine(embeddings, token, n)
	
	# Run PCA on the embeddings
	pc1, pc2 = run_pca(closest_embeddings)
	
	# Create a scatter plot of the embeddings
	plt.figure(figsize=(10, 10))
	plt.scatter(closest_embeddings[:, 0], closest_embeddings[:, 1])
	
	# Add the token text to the plot
	for i, token in enumerate(closest_tokens_vector):
		text = tokenizer.decode(int(token))
		plt.text(closest_embeddings[i, 0], closest_embeddings[i, 1], text)
	
	# Add the principal components to the plot
	plt.arrow(0, 0, pc1[0], pc1[1], color='r', width=0.01)
	plt.arrow(0, 0, pc2[0], pc2[1], color='g', width=0.01)
	
	plt.xlabel('Principal Component 1')
	plt.ylabel('Principal Component 2')
	plt.title('Closest Tokens by Cosine Similarity')
	plt.show()


Todo....

## Playground

A few incomeplete ideas.

### infinite crafter

Try to make something like https://neal.fun/infinite-craft/

In [ ]:
def craft_combine_substances(substance_a, substance_b):
	prompt = f"In one word describe the substance I get when I mix {substance_a} with {substance_b}."
	output_len = 20
	result = model.generate(prompt, device, output_len=output_len)
	return result

In [ ]:
print(craft_combine_substances("fire", "dirt"))
